<h1> <center> Finishing Houdini Framework using $\texttt{Estra}$ results </center> </h1>

*Once the template shading network is created in Houdini using fillcolorramp.py, we are ready to transform our SPH dataset into sphere sprites and create our own custom shader network, shown below. Then, we can copy our ML-informed color ramp into our custom built shading network, render the scene, and create a simple, yet effective visualization of the data.*

Here are two useful resources on building shading networks:
+ http://www.sidefx.com/docs/houdini/shade/build.html
+ http://ytini.com/tutorials/tutorial_moreAboutShaders.html

Lastly, these steps outlined below pertain to Houdini FX 17.5.173, but should still work on previous editions, like Houdini 16.

# Making sphere sprites

+ Go to /obj. To make a *Geometry* node, press *TAB* -> Geometry, and place in the network view. Name it something like "spheresprite_emissive". 
+ Click on the *Geometry* node to highlight it. Above in the parameters box, you will see tabs for "Shading", "Sampling", "Dicing", and "Geometry". Go to the "Geometry" tab and select "Backface removal". This is so that the side of the object not facing the camera is not rendered, to save on computational costs.
+ Double click on spheresprite_emissive to dive into the node network. It should be empty, but we want to fill it with a sphere, so each particle is represented as a sphere, and we can shade/apply properties to these spheres to get the visualization we desire. To do so, click *TAB* -> Sphere and drop it in the network. Change the Primitive Type from "Primitive" to "NURBS".

# Making an instance node with pre-processing

+ Go back to /obj by pressing the "u" key (jumping up a directory). To make an *[Instance](https://www.sidefx.com/docs/houdini/nodes/obj/instance.html)* node, press *TAB* -> Instance, and place in the network view. Name it something like "synestia_spheresprites_emissive".
+ Click on the *Instance* node to highlight it. Above in the parameters box, you will see tabs for "Transform", "Instance", "Render", and "Misc". Go to the "Instance" tab. Make "Instance Object" parameter "/obj/spheresprite_emissive/". Change "Point Instancing" parameter from "Off" to "Fast point instancing". 
+ Dive into the *Instance* node and make an *Object Merge* node. Rename it "IN_SPRITE_POINTS". Make "Object 1" parameter "/obj/synestia/OUT_SPRITE_POINTS/".
+ We preprocessed this particular dataset by thresholding smoothLen > 90 and density > 3.4. Thus, create two *Delete* nodes. Set "Entity" to "Points", and change "Operation" to "Filter by Expression". In "Filter Expression", write "@smoothLen > 90" in the first node and "@density > 3.4" for the second node. Connect the **output** of the *IN_SPRITE_POINTS* node to the **input** of the first *Delete* node, and connect the **output** of the first *Delete* node to the **input** of the second *Delete* node. This is important such that the exact order of the particles in the scene file matches the particles clustered by the algorithms, such that the IDs match to the correct particles in Houdini. 
+ Create an *Attribute Create* node named "pscale_from_smoothLen". Make the "Name" parameter "pscale" and change the "Value" to "@smoothLen * 1" (Houdini natively recognizes an attribute called pscale which automatically scales the size of your particles in the render). Connect the **output** of the second Delete node to the **input** of pscale_from_smoothLen.
+ Make a *Material* node. Name it something like "varsToShader". This makes attribute values available to use in surface shader. Change "Attributes" parameter from "Primitive Attibutes" to "Point Attributes"; change "Material" param to `chsop("../shop_materialpath")` (be sure to include the grave accents) and select "Overrides use local variables". Go to Local Overrides param, click "+". [temperature, float, @temperature]; [density, float, @density]; [pscale, float, @pscale], [id, Integer Value, @id], [position3D, 3-Tuple Vector, @P.x, @P.y, @P.z]. Obviously these may change depending on your attributes in the Geometry Spreadsheet.
+ Make a *Null* node. Connect and rename "OUT_EMISSIVE_SPRITES". Hit the blue display flag on this node and jump up to /obj. This is what will be displayed in the Scene View when the display flag on synestia_spheresprites_emissive is selected. The view in this case looks strange as the pscale attribute allows the particles to be scaled by their size, and in this simulation the particles on the outer fringes are large and dominate the scene view. We will account for this in the shader such that these large particles are mostly opaque. 
+ The network view of synestia_spheresprites_emissive will look like the following:

![](Estra_shader_method/synestia_spheresprites_emissive_network_view.png)

# Making your own simple but powerful custom shader 

*My custom built shader for the Synestia dataset, named "simple_synestia_shader", can be seen within the **synestia_vol.hipnc** file, located in the [Estra Github repository](https://github.com/patrickaleo/estra). I strongly encourage you to use it as a reference when making your own, or edit it as needed to fit your simulation.*

This custom shader network will be built piece by piece. Once these pieces are connected together this will give the bland particles dataset the cinematic, powerful look that will aid in the user's understanding, is of publication quality, and is perfect for public outreach material. Most of the process should be applicable to any SPH astrophysical data visualization. Only certain attribute values may change dataset to dataset. 

## I. Getting ready 

+ In /shop, create a *Material Shader Builder* node. This will be named vopmaterial1, but rename it something like "simple_synestia_shader". Go into the network.
+ Delete the *Global Variables* node named "displacement_globals", as this deals with a *Displacement* Context Type. We care about a *Surface* Context Type, and so we will use the *Global Variables* node named "surface_globals" only.
+ Move surface_globals to the upper-left to give you lots of workspace.
+ Create a *Classic Shader Core* node. It will create with it a node named computelighting1 and connect itself automatically. Within classicshadercore1, go to the Diffuse tab and deselect "Enable Diffuse". In the "Base Reflection" tab, deselect "Enable Base Reflections". In the Emission tab, select "Enable Emission". On its interface, you can click the white arrows to expand all the available inputs. In this example we only care about Emission and Opacity.
+ Next to emit_clr, create a *Multiply* node. Rename it something like "emitMultiply". Connecting the output **product** to **emit_clr** now will throw an error, so we will wait to connect this towards the end of our setup. Also towards the end of this process we will have several **inputs** (once you connect them to input 1, an input 2 will automatically be created and so on...), but this will be done step-by-step. 
+ Click on this *emitMultiply* node and middle click on an **input**. Select "Promote Parameter". Then click on the stub to highlight it and change the "Name" parameter to "emitMult" and change the label from "Input Number #" to "Emission Multiplier". (This will allow you to change the "Emission Multiplier" value in the simple_synestia_shader parameters without having to go into the network, which is where we are now.) In the the simple_synestia_shader parameters, make sure to set "Emission Multiplier" and "Opacity Multiplier" to 1 (if the value is 0, you'll get a black screen when you render).
+ Next to **opac_clr**, create a *Multiply* node. Rename it something like "opacMultiply". 
+ The network should look something like this (the surface_globals node is off screen).

![](Estra_shader_method/getting_ready.png)

## II. Temperature and Density

1. To keep each segment easy to find and compartmentalized, click the "Create network box" icon in the toolbar (it's directly left of the yellow sticky note). Or just press Shift+O. Move this to the left of both *Multiply* nodes and click on it so it has a yellow outline and click on the color pallette, which can also be found in the toolbox. Make this an easily identifiable color like yellow. Next to the (-) sign, name it "Temperature and Density".

1. Create a *Parameter* node. (For clarity, note that parm1 is the default node name, and parm is the Name parameter. Yes it is confusing...) Change the "Name" entry from "parm" to "density". This name needs to match the desired attribute in your geometry spreadsheet (so it can be read in and used), and what you have read into your varsToShader node in /obj. Click to enable the "Invisible" box, so that you don't accidentally set a unique value for density in the shader, but use the range of density data available in the dataset. 

1. To the right of this density param node create a *Fit Range* node. Name it something like "fitDensity". Connect **density** to **val**. Middle mouse click on srcmin and select "Promote Parameter". Do this for srcmax as well. (This will allow you to change the "Minimum Density" and "Maximum Density" values in the simple_synestia_shader parameters without having to go into the network. Although this step isn't required, it's quite useful, especially for prototyping your visualization.) 
    1. Click on srcmin and change the label from "Minimum Value In Source Range" to "Minimum Density", so that in the parameters you will know what this label pertains to. Do likewise for the maximum density for srcmax. You should see the "Source Min" and "Source Max" in this fitDensity node to now be greyed-out. Next to the *Fit Range* node create a *Ramp Parameter* node.

1. For the *Ramp Parameter* node, change the "Name" parameter from "ramp" to "densityRamp" and make the Label "Density Ramp". Change the "Ramp Type" from "RGB Color Ramp" to "Spline Ramp (float)". This is because density does not play a role in the color temperature of the particles-- only how transparent certain particles are (opacity). Connect **shift** of fitDensity to **input** of densityRamp. Connect the output **densityRamp** to the first **input** of both *Multiply nodes*, emitMultiply and opacMultiply, so that the visualization is not black and white. Be warned, if you simply connect **densityRamp** to the first input, it will destroy the emitMultiplier promoted parameter. So you'll first want to connect **densityRamp** to the next available input and highlight the *emitMultiply* node. Use the blue up arrows to move the **temperatureColorRamp** to the first input. The specific placement of **temperatureColorRamp** being the first input of the emitMultiply node matters, but the ordering of the other inputs will not.

1. Repeat steps 2 through 4 but for the new parameter "temperature". Click to enable the "Invisible" box for temperature in the *Parameter* node. Name the *Ramp Parameter* node something like "tempOpacRamp", and add the Label "Temperature Opacity". 

1. Lastly, make a second *Ramp Parameter* node. Change the "Name" parameter from "ramp" to something like "temperatureColorRamp" and make the Label "Temperature Ramp". Connect **shift** of fitTemperature to **input** of temperatureColorRamp. Keep the "Ramp Type" as "RGB Color Ramp", because this ramp will determine the emissive color temperature of the visualization. And because this is emissive, we only want to connect the output **temperatureColorRamp** to an **input** of the emitMultiply node, NOT the opacMultiply node.

1. Now that we have some inputs to the multiply nodes, connect the **product** of emitMultiply to **emit_clr** and the **product** of opacMultiply to **opac_clr**. Once these steps are completed, this section of shading network should look like this:

![](Estra_shader_method/temp_and_density_network.png)

In the $\texttt{Estra}$ notebook we created several template shaders with unique color maps/ramps each corresponidng to the particular machine learning clustering algorithm (K-means, DBSCAN, GMM, etc.) This was done so we can copy these ramps onto temperatureColorRamp without having to redo the fillrampcolor steps over and over again as we test different color mappings for render.

Once the network is created as above, jump up to /shop and select a clustering algorithm shader. For example, this example using a GMM shader named "GMM_shader". Use copy network params option to paste the color ramp data into your shader. Or, just redo the fillcolorramp steps referencing this shader. 

Thus, our 5 cluster GMM color ramp is
![](Estra_shader_method/temperature_ramp.png)

Also, if we want to render the simulation colored by the cluster IDs from a clustering algorithm, we do the same as above, except our idTempRamp has Ramp Type "RBG Color Ramp", Color Type "RBG", and Default Interpolation "Linear". Then, in /shop, we set our id Temp Ramp takes our newly created attribute "map_id_to_clus" and assigns the matching color from our plots. We set Min Id to Clus is 1, Max Id to Clus is 5, and so a position of 0-0.2 (gold) is cluster ID 1, a position of 0.2-0.4 (magenta) is cluster ID 2, etc. Our setup looks like this:
![](Estra_shader_method/clus_id_temp_ramp.png)
![](Estra_shader_method/clus_id_network_box.png)

## III. Pscale / Smoothing Length

1. Although not required, it is useful to create a network box above the one for Temperature and Density. Give it a unique color and name. For this example, I chose to make the box green and name it "pscale/smoothing length".
1. Create a *Parameter* node. Change the "Name" from "parm" to "pscale". In our original dataset we didn't have a pscale attribute in our geometry spreadsheet. This was the purpose of the *Attribute Create* node named "pscale_from_smoothLen" in "synestia_spheresprites_emissive", where we created the pscale attribute from smoothLen. This is the parameter to be read in to control the particle radius size (pscale).
1. To the right of this density *Parameter* node create a *Fit Range* node. In this example, I changed the parameter "Source Max" from "1" to "100", because in my geometry spreadsheet the range of values of pscale is roughly 0 to 100. This will change between datasets. Name it something like "fitPscale". Connect **pscale** to **val**. In this example there was no need to promote srcmin and srcmax. 
1. As before, create a *Ramp Parameter* node. Connect **shift** of fitPscale to **input** of pscaleOpacityFade. Change the "Name" parameter from "ramp" to "pscaleOpacityFade". Change the "Ramp Type" from "RGB Color Ramp" to "Spline Ramp (float)". This ramp will determine how transparent particle will be based off of its size. For example, in this simulation the largest particles (largest pscales close to 100) are on the outer fringes; most of the partcles that constitute the core are very small (close to 0). When making these particles spheres, the largest particles will dominate the visulization and that is all you will see, unless the largest particles are very transparent. How the transparency/opacity depends on the size is determined by the user via the ramp.
1. Connect **pscaleOpacityFade** to **input** of both the emitMultiply and opacMultiply.

![](Estra_shader_method/pscale_network.png)

## IV. Checking Our Progress: Setting Ramp Values and doing a Test Render

*To see if we have a working shader and to prove to ourselves that we are making progress, let's set our ramp values and do a test render. It will not look like a finished product quite yet, but it will look a lot better than an agglomeration of grey particles!* 

1. When in simple_synestia_shader, press the "u" key to jump up to the /shop tab. Click to highlight simple_synestia_shader, and from there we can edit our parameters and ramps. These specific values will change dataset to dataset, but the process should be the same, especially the line of thinking.
    1. In this example, there are an overwhelmingly a lot of particles in the hot, dense core. Because of this, it will be quite expensive to render, especially locally on a single CPU, and in the end will be indistinguishable from another render that has a still substantial but overall far fewer particles in the core. Because all the most dense particles are in the center, we will threshold density by setting the "Maximum Density" to 2.25 instead of 3.3999 in the dataset (you can check this by looking at the values in the geometry spreadsheet, and clicking on the desired attribute to toggle its order from greatest to least and vice-versa. You need to be in /obj to do this.). This means all the particles with densities > 2.25 will be neglected when rendering. The reason of doing this and not changing the density value in THRESHOLD_Density in synestia_spheresprites_emissive is you can more easily change how you want the data to be available to render than limiting the dataset you read in.
    1. With the densest particles constituting the core, the lighest particles (of which there are many) are spread all throughout the simulation. However, this means that if the least dense particles are very bright, they will be all that is seen, even making it appear as if there is no core. Thus, when modifying the "Density Ramp", we need to find a good balance between allowing the densest particles to be seen in the core without drowning out the least dense particles and vice versa. Here, we set the "Density Ramp" at "Position 0" to have a "Value" of "0.03". Note: the higher the value the more visible in the image that particular density of particles (normalized on a 0-1 scale), which means that the few, most dense particles are as bright as they can be, and the many, least dense particles are individually not so bright, but it will balance out as we'll come to see in the final render. Keep in mind these values should be tuned to best fit your dataset. For this dataset, I have assumed the density ramp to have the form of a Cubic Bezier Curve, governed by the equation: $$\textbf{B}(t)=(1-t)^3 \textbf{P}_0 + 3(1-t)^2 t \textbf{P}_1 + 3(1-t)^2 t^2 \textbf{P}_2 +t^3 \textbf{P}_3, 0 \leq t \leq 1,$$ with control points $\textbf{P}_0 = (0,0)$... (to match the rectangular box of the ramps which are [0,1] in x and [0,1] in y). Normally the "Position" at "0" would have a "Value" at "0", but this would make the smallest particles completely invisible, so we have to give it a small, non-zero value. However, all the other values follows this form; "Position 0.1" has a point with "Value" of "0.53", "Position 0.25" has a "Value" of "0.75", "Position 0.5" has a "Value" of "0.91", "Position 0.75" has a "Value" of "0.98", and the point at "Position 1" has a "Value" of "1". Then, make sure Interpolation is changed from "Linear" to "Bezier" for all points (changing it only does it for one point. You need to highlight each point individually and change it). When plotting a histogram of the number of particle counts as a function of binned density, there is a double peak: one at the lowest density, and one at the highest density, where after the first peak there is steep decline and gradual increase thereafter until the second peak. ![](figures/rho_histogram.jpg) Thus, this cubic bezier curve is a good starting point in allowing a particle to be more opaque and emissive (as this density ramp is connected to both opacMultiply and emitMultiply) with increasing density, while allowing for the many contained lowest density particles in the core to be visible and not overpower the visualization. It is expected that this Cubic Bezier Curve will be a good assumption for SPH datasets, particularly of high impact simulations. This density ramp will look like the following: ![](Estra_shader_method/density_ramp_bezier_curve.png)
    1. According to the data in the geometry spreadsheet, the minimum temperature is 2343.28 and the max is 14308.5. So let us set these values to "Minimum Temperature" and "Maximum Temperature", respectively, in our shader. The "Temperature Opacity" ramp should be set to a Value of 1 for the entire range. This simply means that opacity will not be dependent on temperature. Because each particule has density and temperature, it would be unnecessary and more contrived to have both attributes affect opacity.
    1. For "Temperature Ramp", this is what our clustering results from Estra spit out. In /shop you should have different shaders with temperature maps for each of the different clustering algorithm results. Pick the best one and you can right click on the name of the ramp and say "Copy Parameter" and go to temperature ramp in simple_synestia_shader and right click and select "Paste Relative References". (Or you can use the fillrampcolor.py with the specific name of your temperature ramp as the appropriate input. But pasting relative parameters will be useful if you are testing out different clustering results and are not yet committed to one.) You should see a nice color map such as this (for GMM): ![](Estra_shader_method/temperature_ramp.png)
    1. Last is pscaleOpacityFade. This ramp controls how the relative size of each particle affects the opacity (and thus, transparency). For this example, our largest sized particles are in the outer periphery of the simulation, and so after rendering these particles will dominate and overshadow the entire visualization. Hence, we want a ramp that shows the smallest particles (high opacity = low transparency), and after a certain size falls off rapidly (low opacity = high transparency). In this example we also used a Cubic Bezier curve, but took the mirror image of the one used for the Density Ramp. Here, "Position 0" has a "Value" of "1", "Position 0.25" has a "Value" of ".98", "Position .5" has a "Value" of ".91", "Position 0.75" has a "Value" of "0.75", "Position 0.9" has a "Value" of "0.53", and "Position 1" has a "Value" of "0.3". (There is no discernable difference if the "Position 1" has a "Value" of "0" instead). ![](Estra_shader_method/pscaleOpacityFade_Bezier_Curve.png)
    1. Don't forget to set "Emission Multiplier" and "Opacity Multiplier" to 1 to start!
    1. Now double check that your Instance in /obj (synestia_sphereSprites_emissive) under the Render tab is referencing your shader (/shop/simple_synestia_shader/). Select the Render View Tab in the top left of the Houdini GUI and change "ROP Camera" to "Scene View 1" and click the Render button. You may have to adjust the scene view to get a better render orientation. (Obviously, if you create a specific camera, you can change the render view to that specific camera).
    
![](Estra_shader_method/midway_render.jpg)

This test render (rendered to only 5% completeness for time) looks better than a grey clump of particles, but we still have a bit more to go. But visually we can see from the color mapping that we have a redder, cooler outer region and a hotter, whiter/bluer inner core region.

## V. Screen Space Distance From Center

1. In a new network box, we will create a section for calculating the screen space distance from the center. Make this network box a different recognizable color. Here I chose dark grey.
1. Create a Global Variables node. Select "Output A Single Variable". For "Variable Name" select "Surface Normal N".  Change the name from global1 to N_global. Next to it on its right, create a "Normalize" node. Rename it to N_normalize. Connect output **N** of N_global to input **vec** of "N_normalize". 
1. Below these two nodes, repeat steps (1) and (2) for the varible **I -- the Direction from Eye to Surface**.
1. Create a *Dot Product* node. Rename it something like "NdotI". Set the "Signature" parameter from "3D Vector" to "3D Normal/Normal". Connect the **nvec** output of N_normalize to the first input **vec1** of NdotI and connect the **nvec** output of I_normalize to the second input **vec2** of "NdotI".
1. To the right of NdotI, create another *Multiply* node. Name it "multiplySelf". Connect the **dotprod** output of NdotI to both inputs (**input1, input2**) of multiplySelf. 
1. To the right of *multiplySelf* node, create a *Subtract* node. Next to this node, create a *Constant* node. Change the parameter value of "1 Float Default" from "0" to "1". Change the "Constant Name" to "input1". Connect the **input1** output of the "Constant" node to **input 1** input of "Subtract" node. Connect **product** output of multiplySelf to **input2** of *Subtract* node.
1. To the right of the *Subtract* node, create a *Square Root* node. Connect the **diff** output of *Subtract* node to **val** input of *Square Root* node.

![](Estra_shader_method/screen_space_distance_network.png)

## VI. Gaussian Falloff

1. We will use the screen space to center distances to tailor an appropriate Gaussian falloff profile for each sphere (particle), where we will code in Houdini the standard Gaussian equation $$f(x)=a\mathrm {e} ^{-(b-x)^{2}/(2c^{2})},$$ where the parameter $a$ is the height of the curve's peak, $b$ is the position of the center of the peak and $c$ (the standard deviation) controls the width of the "bell". The form of a probability density function of a normally distributed random variable with expected value $\mu$ = b and variance $\sigma^2 = c^2$ is $$g(x)={\frac {1}{\sigma{\sqrt {2 \pi}}}}\mathrm {e} ^{-\frac{1}{2}((x-\mu)/\sigma)^2}.$$

1. Build another network box next to the Screen Space Distance from Center network box. Name it something like "Gaussian Falloff". Create a *Multiply* node. Connect the output **sqrt** of the *sqrt1* node to **input1** and **input2** of this multiply node. This represents the $(b-x)^{2}$ component in the Gaussian equation. It may be helpful to rename this multply node "b-x_squared" so you don't forget what this is doing (carrots, parentheses, and other keystrokes appear as underscores in the node names).

1. Next create a *Divide* node. Connect the **product** of the "b-x_squared" multiply node to **input1** of the *Divide* node. For **input2**, we need to create the denominator $(2c^{2})$. To do so, find ample space under the mulitply node and create a *Parameter* node. Change the "name" from "parm" to "bellWidth", and add in "Bell Width" for the "Label" entry. This will allow you to control the value of $c$ in the /shop level for "simple_synestia_shader". Below this, create a *Constant* node. Change the "1 Float Default" from a value of "0" to "2". Then, to its right, create another *Multiply* node. Connect the **bellWidth** output to **input1** and **input2** of this new multiply node, and connect the **Value** output of the *Constant* node to **input3**. Now this multiply node is the desired  $(2c^{2})$ component. Feel free to rename this node "2c_squared". Finally, to make this the denominator of our exponential, connect the **product** output to **input2** of the *Divide** node.

1. Next to the *Divide* node, create a *Multiply Constant* node. Change the "Multiplier" from "1" to "-1". Connect **div** of *Divide* node to the input **val** of the *Multiply Constant* node. 

1. Next to the *Multiply Constant* node, create an *Exponential* node. Connect **scaled** output of *Multiply Constant* node to input **val** of *Exponential node*.

1. Next to the *Exponential* node, create a *Multiply* node. Connect the output **exp** of the *Exponential* node to **input1** of the new *multiply*. For our penultimate step, create a new *Parameter* node. Change the "name" from "parm" to "bellPeak", and add in "Bell Peak" for the "Label" entry. This will be the $a$ value in our above general Gaussian expression. Connect the **bellPeak** output to **input2** of the *multiply* node. Lastly, to the right of this *multiply* node, create a *Null* node. Name it something like "FALLOFF_PROFILE" (the all-caps allows it to be easily found in searches), and you can change the color too. Connect the **product** output of the *multiply* node to the **next** input of the "FALLOFF_PROFILE" node (this will change it to **product**.) And now, connect the **product** output of the "FALLOFF_PROFILE" node to the available **input5** of the "emitMultiply" node and the available **input5** of the "opacMultiply" node.

1. Congratulate yourself for implementing a Gaussian falloff profile!

![](Estra_shader_method/gaussian_falloff_network.png)

## VII. Another Test Render

To see our great work, let's do another test render to see how it looks. In /shop with simple_synestia_shader, set "Bell Width" to a value of "0.22" and Bell Peak to "1", and then render the scene as we've done before:

![](Estra_shader_method/screen_space_gaussian.jpg)

This test render (rendered to only 5% completeness for time) looks much closer to a finish render. But we don't see much clumping of the dust and gas clouds. So we need to add some randomized noise among some other things to get our desired look.

## xx. Orient Spheres Toward Camera (only needed for dome renders)

*Computer graphics are 2D images. However, our simulation is in 3D space. So we need to convert our 3D data and project it onto a 2D plane, a process called rendering. Essentially, you take the particles in model space and, because you know how this relates to world space (the physical space that your simulation exists in), you perform this mapping. Then, you place a camera in the world, and you map from world space to camera space-- the coordinate system relative to the camera. Camera space is where the particles appear to be before the projection is done for the render.*

*In this section, we orient the sphere sprites towards the camera by taking the particles in model space and orienting them so that from the camera's POV, they face the camera head-on.*

1. Next to the long forgotten "surface_globals" node, create a large network box and name it something like "Orient Spheres Towards Camera". Also change the color if you like (in this example this network box is red). In this network box, create a *Transform* node. Connect the **P** (Position) output of "surface_globals" to the **from** input on this *Transform* node. In the *Transform* node, change the "To Space" parameter entry from "space:current" to "space:camera" (Camera Space in the drop-down menu). What this does is change the source geometry (the particles) from object space (the coordinate system relative to the objects) into camera space (the coordinate system relative to the camera) using a transformation matrix.

1. To the right of *Transform*, create a *Subtract* node. Connect the **to** output of *Transform* to **input1** of *Subtract*. Above the *Transform* node, create a *Parameter* node. Change the "Name" entry from "parm" to "position3D", change the "Type" from "Float (float)" to "3 Floats (vector)", and click to enable the "Invisible" box. Make sure the "Name" entry matches verbatim the Parameter Name entry you had for the varsToShader in /obj/synestia_sphereSprites_emissive so that they are linked. This will read in the x,y,z position values in the geometry spreadsheet for each particle. 

1. As before, to the right of this create a *Transform* node. Connect the **position3D** output of the *parameter* node to the **from** input on this *Transform* node. In the *Transform* node, change the "To Space" parameter entry from "space:world" to "space:camera". It will also help to change this node's name to something like "transform_pos3D". Connect the **to** output of the *transform* node to **input2** of the *Subtract* node.

1. Next to the *Subtract* node, create a *Divide* node. Connect **diff** output of *Subtract* node to **input1** of the *Divide* node. For **input2** of *Divide*, create a *Parameter* node for "pscale", as we've done before. Enable the Invisible option, as you want to use the pscale data from the simulation, and not manually entering in a unique pscale value.

1. Next to this *Divide* node, create a *Multiply* node. Connect the **div** output of *Divide* to **input1** of the new *Multiply* node. For **input2** we will need to backtrack first.

1. Next to the "transform_pos3D" node, also create a *Look At* node.
    > A Look At camera matrix in computer graphics defines two points using an "up" vector: a reference "from" point defining the world position of the camera (eye) doing the observing of the scene, and a target "to" point (the point in space we want to look at, which in this case is the 3D position points of each particle).

1. For the **from** input, create a *Constant* node and select the "Constant Type" to be "3 Floats (vector)". Keep the values (0,0,0). Connect the **Value** output of the constant Node to the **from** input of the *Look At* node.

1. For the **up** input, create a *Constant* node and select the "Constant Type" to be "3 Floats (vector)". Change the values to (0,1,0). Connect the **Value** output of the *Constant* node to the **up** input of the *Look At* node.

1. Connect the **to** output of the "transform_pos3D" node to the **to** input of the *Look At* node. Now we have defined the appropriate Look At transformation matrix that will orient the scene the properly.

1. Next to the *Look At* node, create a *Transpose* matrix. Connect the **matx** output of the *Look At* matrix to the **val** input of the *Transpose* matrix. 

1. Now we can connect the **transpose** output to **input2** of the multiply node. This completes the "Orient Spheres Towards Camera" network box. For reference, it should look something like the following:

![](Estra_shader_method/orient_spheres_camera_network.png)

## VIII. Randomize sprite rotation

1. Above the "Orient Spheres Towards Camera" network box, create one for "Randomize Sprite Rotation". Make it a distinguishable color (for this example I chose light brown). 

1. Create a *Parameter* node for "id". Again, ensure this name matches the "id" name you chose in varsToShader ([id, Integer Value, @id]) so that Houdini will use the id values in the dataset (if the dataset doesn't label particles with id numbers, you can create them.). Select the Invisible box.

1. Next to *Parameter* node, create an *Inteteger To Float* node. Connect **id** to **ival**. Next to the *Integer to Float* node, create your last *Multiply node*(!). Connect **fval** to **input1**. For **input2** create a *Constant* node and make the value some fairly large number like 1000. Connect **value** to **input2**.

1. Next to *Multiply*, create a *Float to Vector* node. Connect the **product** output to the **fval3** input, not **fval1** or **fval2**. This will make the vector <0,0,product>, which will change how much on the z-axis the objects will rotate. We will turn this vector into a quaternion.
    > Quaternions are 4D complex numbers with one real axis and three imaginary axes. They are effectively an angle-axis representation of orientation. Thus, you can perform rotations with quaternions.
    
1. To turn this vector into a quaternion, create a *Vector to Quaternion* node and connect **vec** output of our *Float to Vector* node to the **vec** input of our *Vector to Quaternion* node. Now our network box should look like the following:

![](Estra_shader_method/randomize_sprite_rotation_network.png)

## IX. Noise

1. Create a final, last new network box. Call it something like "Noise" and give it a distinctive color (I chose purple). Create a *Rotate By Quaternion* node and connect the **quat** output of the *Vector to Quaternion* node in "Randomize Sprite Rotation" to the **quaternion** input of the *Rotate By Quaternion* node. Connect the output **product** of the *Multiply* node in "Orient Spheres Towards Camera" to the **vec** input of the *Rotate by Quaternion* node.

1. Next to the *Rotate By Quaternion* node create an "Anti-Aliased Noise" node. Connect the output **result** of *Rotate By Quaternion* to the **pos** input of the "Anti-Aliased Noise" node. For **freq** input, create a *Parameter* node, name it "freq" and put the label as "Frequency", so you can adjust the value in the parameters of simple_synestia_shader in /shop. If you change the type to "3 Floats (Vector)", you can control the frequency along the x,y,z components. 

1. For **amp** input, create a *Parameter* node, name it "amp" and put the label as "Amplitude". Set the "1 Float Default" value to "1" and the "Float Range" values to "-1" in the left-hand column and to "1" in the right-hand column.

> To better randomize the noise, you can add another network box to randomize the sprite noise patterns and have the output of that network feed into **offset**, but the final render doesn't look any better or worse without it, so we will ignore this for this simpler example.

1. Promote the parameters for **rough**, **maxoctave**, and **noisetype**, so these values can be set above in /shop in simple_synestia_shader.

1. Next to *Anti-Aliasing Noise* node, create a *Fit Range* node and connect **noise** output to **val** input. Set "Source Min" value to "-0.5" and "Source Max" value to "0.5". 

1. Next to the *Fit Range* node, create a Null node and name it something like "OUT_NOISE_PATTERN". Next to "OUT_NOISE_PATTERN", create two *Fit Range* nodes. Name one something like "fitOpacityNoise" and the other "fitEmissionNoise". Connect the **shift** output of "OUT_NOISE_PATTERN" to the **val** inputs of both "fitOpacityNoise" and "fitEmissionNoise". For both nodes, promote the **destmax** parameter. Name the Label "Opacity Noise Strength" and "Emission Noise Strength".

1. Lastly, connect the **shift** output of "fitOpacityNoise" to an **input** of the "opacMultiply" node, and connect the **shift** output of "fitEmissionNoise" to an **input** of the "emitMultiply" node. This is so you can control the individual noise separately, if you find it better suits your visualization.

1. This last network box should look like this:

![](Estra_shader_method/noise_network.png)

## X. Setting Some Parameter Values

1. The last step now is to try and find the best values for your visualization. Some of these may be chosen to highlight a certain feature, to get across a certain point for educational purposes, etc. The goal of $\texttt{Estra}$ is to have some of these values influenced or informed by machine-learning, as the temperature ramp is.

1. Go to /shop and highlight simple_synestia_shader. For this example, we set Frequency to 3.5 for x,y,z directions, Amplitude to 1.25, Roughness to 0.8, Max Octaves to 12, Noise Type to Perlin, Bell Width to 0.22, Bell Peak to 1, Opacity Noise Strength to 1 and Emission Noise Strength to 1.

1. With this, let's do our final render! Our final setup looks like this
![](Estra_shader_method/full_network_setup.png)

## XI. The Final Render!

![](Estra_shader_method/simple_synestia_shader_4k.jpg)

Note: The above is the same scale and aspect ratio as our midway test render. But this contains too much extra Gaussian falloff and is zoomed out too much from our areas of interest.

The below is of our final rendering of the true synestia.

![](final_renders/Estra_GMM_5clus.jpg)

How does this compare to the AVL version of the same dataset? Take a look below. This was acheived using a much more complicated network.

![](final_renders/AVL_final.jpg)

![](final_renders/AVL_final_render_1%.jpg)

## XII. Final Thoughts

As we can see, the $\texttt{Estra}$ rendering is quite close to the AVL rendering. And in all, most of the values are make with simple assumptions (Bezier curves influencing density ramp, pscale opacity ramp), or are informed from the clustering algoritms (temperature ramps). With a simple shader network that can work with a range of SPH astronomical datasets, this is a good method for any scientist or interested user to foray into cinematic astrophysical data visualization. Once these visualizations are produced, they can easily be modified to attune and satiate whatever desire/purpose is best. These visualizations can be used for scientific outreach and communiction, publications, simulation prototyping, and other avenues. Overall, we hope that the science community can benefit from understanding visualizations, and there can be more communication between visualization artists/teams and domain scientists.